In [1]:
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install rich[jupyter]

# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 12.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 12.0 MB/s 
     |████████████████████████████████| 6.6 MB 52.2 MB/s 
     |████████████████████████████████| 101 kB 9.6 MB/s 
     |████████████████████████████████| 596 kB 50.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 232 kB 10.7 MB/s 
     |████████████████████████████████| 51 kB 6.1 MB/s 


In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
tokenizer = T5Tokenizer.from_pretrained('google/mt5-small')
model = T5ForConditionalGeneration.from_pretrained('google/mt5-small')
model = model.to(device)

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


In [48]:
text = "Question: 안녕하세요. 오늘은 날씨가 참 좋네요."

In [20]:
text_tokens = tokenizer.encode_plus(text).input_ids

In [21]:
tokenizer.decode(text_tokens)

'안녕하세요. 오늘은 날씨가 참 좋네요.</s>'

In [23]:
tokenizer.convert_tokens_to_ids(text)

2

In [29]:
input_ids = tokenizer.encode("summarize: Hello, my dog is cute", return_tensors="pt").to(device)
outputs = model.generate(input_ids)
outputs

tensor([[     0, 250099,      1]], device='cuda:0')

In [53]:
import warnings
warnings.filterwarnings('ignore')
inputs = tokenizer.prepare_seq2seq_batch(src_texts=text,
                                          return_tensors='pt',
                                          max_length=128).input_ids.to(device)
output = model.generate(inputs,
                    max_length=200,
                    repetition_penalty=2.0,
                    early_stopping=False,
                    num_beams=10)
tokenizer.decode(output[0])

'<pad> <extra_id_0>.</s>'